In [1]:
import psycopg2
import pandas as pd
import re
import random

In [2]:
db_name="deep_diver_db"
db_endpoint="deep-diver-db.cvulnxnfevj3.us-east-1.rds.amazonaws.com"
user="deep_diver"
password="Amazon2022"

In [3]:
def get_data(tablename:str,conn=None,):
    if not conn:
        conn = psycopg2.connect(
            host=db_endpoint,
            database=db_name,
            user=user,
            password=password)  
    return pd.read_sql(f"SELECT * FROM {tablename}", conn)

In [4]:
def create_top_10_recipes():
    df_recipe_trend = get_data(tablename="recipe_search_trend")
    df_recipe_trend['date_last7d'] = pd.to_datetime(df_recipe_trend['date_last7d'])
    df_recipe_trend = df_recipe_trend.sort_values(by="date_last7d", ascending=False)
    df_last_date = df_recipe_trend[(df_recipe_trend['date_last7d'] == max(df_recipe_trend['date_last7d']))]
    recipe_search_list = df_last_date.sort_values(by="value", ascending=False).head(10)['query_clean'].tolist()
    
    df_recipe = get_data(tablename="recipes")
    # creating dictionary for each recipe trend word
    recipe_book = {}
    for index, row in df_recipe.iterrows():
        for product in recipe_search_list:
            if not product in recipe_book.keys():
                recipe_book[product] = []
            if product in row['label'] and row['price'] != '0':
                recipe_book[product].append(index)

    #randomly selecting recipes for each trend word
    row_selection = []
    for entry in recipe_book.keys():
        if len(recipe_book[entry]) != 0:
               row_selection.append(random.choices(recipe_book[entry])[0])

    while len(row_selection) != 10:
        entry = random.choices(list(recipe_book.keys()))[0]
        if len(recipe_book[entry]) != 0:
            row_selection.append(random.choices(recipe_book[entry])[0])
            row_selection = list(dict.fromkeys(row_selection))
    df = df_recipe.iloc[row_selection]
    return df.reset_index()

In [5]:
def create_top_10_product():
    df_product_trend = get_data(tablename="product_search_trend")
    df_product_trend['date_last7d'] = pd.to_datetime(df_product_trend['date_last7d'])
    df_product_trend = df_product_trend.sort_values(by="date_last7d", ascending=False)
    df_last_date = df_product_trend[(df_product_trend['date_last7d'] == max(df_product_trend['date_last7d']))]
    product_search_list = df_last_date.sort_values(by="value", ascending=False).head(10)['query_clean'].tolist()

    df_recipe = get_data(tablename="recipes")
    # creating dictionary for each recipe trend word
    product_rows = {}
    for index, row in df_recipe.iterrows():
        for entry in row['ing_quant']:
            for product in product_search_list:
                if not product in product_rows.keys():
                    product_rows[product] = []
                if product in entry:
                    product_rows[product].append(index) 

    #randomly selecting recipes for each trend word
    row_selection = []

    while len(row_selection) != 10:
        entry = random.choices(list(product_rows.keys()))[0]
        if len(product_rows[entry]) != 0:
            row_selection.append(random.choices(product_rows[entry])[0])
            row_selection = list(dict.fromkeys(row_selection))
    df = df_recipe.iloc[row_selection]
    return df.reset_index()

In [6]:
# 10 top product
from sqlalchemy import create_engine, inspect
df = create_top_10_product()
engine = create_engine(
        f'postgresql://{user}:{password}@{db_endpoint}:5432/{db_name}')
with engine.connect() as conn:
    df.to_sql(name="top_product_trend", con=conn, if_exists='replace', index=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
# 10 recipe product
from sqlalchemy import create_engine, inspect
df = create_top_10_recipes()
engine = create_engine(
        f'postgresql://{user}:{password}@{db_endpoint}:5432/{db_name}')
with engine.connect() as conn:
    df.to_sql(name="top_recipe_trend", con=conn, if_exists='replace', index=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
def formating_dataframe_by_colum(df:pd.DataFrame, column:str):
    for index, row in df.iterrows():
        entry = row[column]
        if "{" in row[column]:
            entry = row[column][1:-1]
        entry = entry.strip('\"')
        df.at[index,column] = entry
    
    return df

In [254]:
from sqlalchemy import create_engine, inspect
#df = create_top_10_product()
engine = create_engine(
        f'postgresql://{user}:{password}@{db_endpoint}:5432/{db_name}')
with engine.connect() as conn:
    df_recipe.to_sql(name="recipes", con=conn, if_exists='replace', index=False)

In [ ]:
df_recipe = formating_dataframe_by_colum(df_recipe, column="meal_type")
df_recipe = formating_dataframe_by_colum(df_recipe, column="dish_type")
df_recipe

In [7]:
df_product = pd.read_csv("DWL1/Code_Submission/deep_divers/Coop/master_csv.csv")

In [8]:
df_product.iloc[806]

Unnamed: 0                                                     806
title                                        Cherry Yogurts 2x180g
href             /en/food/dairy-products-eggs/yogurt/flavoured-...
quantity                                                   2x 180g
rating                                                         5.0
price                                                          1.2
oldPrice                                                       NaN
savingText                                                     NaN
category_1                                     dairy-products-eggs
category_2                                                  yogurt
category_3                                        flavoured-yogurt
price per 100                                                  NaN
Name: 806, dtype: object

In [9]:
def unifying_unit(df:pd.DataFrame, unit:str, new_unit:str,calculator=int):

    for index, row in df.iterrows():
        if isinstance(row['quantity'],str):
            if re.search(r'\d+'+unit, row['quantity']):
                if "x" not in row['quantity']:
                    old_value = row['quantity']
                    new_value = int(float(old_value.split(unit)[0])*calculator)
                    df.at[index, 'quantity'] = f"{new_value} {new_unit}"
    return df

In [124]:
df_product = unifying_unit(df=df_product, unit="kg", new_unit="g", calculator=1000)
df_product = unifying_unit(df=df_product, unit="l", new_unit="ml", calculator=1000)
df_product = unifying_unit(df=df_product, unit="dl", new_unit="ml", calculator=100)
df_product = unifying_unit(df=df_product, unit="cl", new_unit="ml", calculator=10)

In [24]:
def price_per_unit(df:pd.DataFrame,unit:str, calculator:int=100):
    for index, row in df.iterrows():
        if isinstance(row['quantity'],str):
            if re.search(r'\d+'+unit, row['quantity']):
                if "x" not in row['quantity']:
                    value_no_unit = int(float(row['quantity'].split(unit)[0]))
                    price_float = float(row['price'])
                    if value_no_unit != 0:
                        new_value = (price_float/value_no_unit)*calculator
                        df.at[index,'price per 100'] = round(new_value,2)
    return df

## Recipe based on diet and product trend

In [17]:
def get_trend_recipes():
    # diet trend
    df_diet_trend = get_data(tablename="diet_search_trend")
    df_diet_trend['date_last7d'] = pd.to_datetime(df_diet_trend['date_last7d'])
    df_diet_trend = df_diet_trend.sort_values(by="date_last7d", ascending=False)
    df_diet_trend_last_date = df_diet_trend[(df_diet_trend['date_last7d'] == max(df_diet_trend['date_last7d']))]
    diet_trend_list = df_diet_trend_last_date.sort_values(by="value", ascending=False)['diet_category'].tolist()
    if 'keto' in diet_trend_list:
        diet_trend_list.append("keto-friendly")
    
    # recipe trend
    df_recipe_trend = get_data(tablename="recipe_search_trend")
    df_recipe_trend['date_last7d'] = pd.to_datetime(df_recipe_trend['date_last7d'])
    df_recipe_trend = df_recipe_trend.sort_values(by="date_last7d", ascending=False)
    df_last_date = df_recipe_trend[(df_recipe_trend['date_last7d'] == max(df_recipe_trend['date_last7d']))]
    recipe_search_list = df_last_date.sort_values(by="value", ascending=False)['query_clean'].tolist()
    
    # product trend
    df_product_trend = get_data(tablename="product_search_trend")
    df_product_trend['date_last7d'] = pd.to_datetime(df_product_trend['date_last7d'])
    df_product_trend = df_product_trend.sort_values(by="date_last7d", ascending=False)
    df_last_date = df_product_trend[(df_product_trend['date_last7d'] == max(df_product_trend['date_last7d']))]
    product_search_list = df_last_date.sort_values(by="value", ascending=False)['query_clean'].tolist()
    
    df_recipe = get_data(tablename="recipes")
    recipe_list= []
    for index, row in df_recipe.iterrows():
        if row['price'] != "0":
            # diet trend
            if "{" in row['health_labels']:
                diet_value = row['health_labels'][1:-1].lower()
                diet_value_list = diet_value.split(",")
                for diet in diet_value_list:
                    if diet in diet_trend_list:
                        recipe_list.append(index)

            # recipe trend
            for recipe in recipe_search_list:
                if recipe in row['label']:
                    recipe_list.append(index)

            # product trend
            for entry in row['ing_quant']:
                for product in product_search_list:
                    if product in entry:
                        recipe_list.append(index)

    final_list = list(dict.fromkeys(recipe_list))
    
    df = df_recipe.iloc[final_list]
    return df.reset_index()

In [10]:
df = get_trend_recipes()

In [18]:
# upload_ recipes lists
from sqlalchemy import create_engine, inspect
df = get_trend_recipes()
df = df.sample(frac = 1)
engine = create_engine(
        f'postgresql://{user}:{password}@{db_endpoint}:5432/{db_name}')
with engine.connect() as conn:
    df.to_sql(name="recipes_trend", con=conn, if_exists='replace', index=True)

### Reproduce Product transformation

In [5]:
df_product = pd.read_csv("DWL1/Code_Submission/deep_divers/Coop/master_csv.csv")

In [8]:
df_test = df_product.loc[:, df_product.columns!='href']

In [12]:
df_test.iloc[0]['quantity'] = "1 kg"

C:\Users\taamavik\AppData\Local\Temp\ipykernel_2180\139541052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.iloc[0]['quantity'] = "1 kg"


In [16]:
df_test.at[1,"quantity"] = "1 kg"

In [29]:
df_product.iloc[:,~df_product.columns.isin(["href","oldPrice","savingText","category_1","category_2","category_3"])]

,title,quantity,rating,price,price per 100
0,Abate Fetel pears ca. 1kg,1000 g,5.0,4.20,0.42
1,Äpfel Envy I ca. 1kg,1000 g,5.0,4.95,0.50
2,Äpfel Gala,750g,5.0,2.70,0.36
3,Apricot ca. 500g,500g,2.5,3.75,0.75
4,Betty Bossi Cubed Watermelon,250g,0.0,3.95,1.58
...,...,...,...,...,...
8265,Ravioli Alla Napoletana,870g,4.5,3.20,0.37
8266,Saitaku Miso Soup 4 Pieces,4PCE,5.0,3.70,0.93
8267,Thai Kitchen Tom Kha Soup Bowl,70g,3.5,3.10,4.43
8268,Thai Kitchen Tom Yum Soup Bowl,75g,3.5,3.10,4.13
